# Heatwaves

## Delve deeper into heatwaves and connect RQ1 and soil moist, and fstr analysis

In [ ]:
import os
datapath = os.path.join('../')
#print('datapath is set to %s'%datapath)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import plotly.express as px
#import cufflinks as cf
import matplotlib.dates as mdate
import matplotlib.ticker as ticker
from matplotlib import cm
#from colorspacious import cspace_converter
import scipy.stats as stats
#cf.go_offline()
#cf.set_config_file(offline=False, world_readable=True)

from datetime import datetime, timedelta
import sys
sys.path.insert(0, os.path.join(datapath,'PythonScripts'))

#from Loobos_Toolbox import dateparse, dateparse_Gapfilled, Read_LoobosEddFinal, Read_LooStor, Read_LoodatGapfill, Read_Loobos_halfhourly, Read_Loobos_meteo, Read_Loobos_soil, Read_Loobos_profile
#from Ags_model import runAgs, calc_LE , calc_LE2

### historical heatwave data table creation 

In [ ]:
# based on a idea/method from Vermeulen, 2015
# list of heat waves from https://www.knmi.nl/nederland-nu/klimatologie/lijsten/hittegolven:
#   
#input=[
#   "'Start' , 'end(inc.)', 'duration(days)' ,'tropic. days','MaxTemp','MaxTempDate','heatwave number'",     #heatwave number* 
#   "'22 aug 2001' ;	'26 aug 2001'; 	5 	;3 	       ;31.1 	;'25 aug 2001' 	;24.0",
#   "'31 jul 2003' ;	'13 aug 2003'; 	14 	;7 	       ;35.0 	;'07 aug 2003' 	;67.4",
#   "'02 aug 2004' ;	'11 aug 2004'; 	10 	;3 	       ;32.5 	;'09 aug 2004' 	;35.1",
#   "'18 jun 2005' ;	'24 jun 2005'; 	7 	;3 	       ;32.8 	;'20 jun 2005' 	;28.5",
#   "'30 jun 2006' ;	'06 jul 2006'; 	7 	;3 	       ;32.0 	;'04 jul 2006' 	;31.7",
#   "'15 jul 2006' ;	'30 jul 2006'; 	16 	;8 	       ;35.7 	;'19 jul 2006' 	;85.8",
#   "'21 jul 2013' ;	'27 jul 2013'; 	7 	;3 	       ;32.6 	;'22 jul 2013' 	;27.8",
#   "'30 jun 2015' ;	'05 jul 2015'; 	6 	;3 	       ;33.1 	;'01 jul 2015' 	;33.5",
#   "'15 jul 2018' ;	'27 jul 2018'; 	13 	;4 	       ;35.7 	;'26 jul 2018' 	;52.7",
#   "'29 jul 2018' ;	'07 aug 2018'; 	10 	;4 	       ;33.9 	;'07 aug 2018' 	;40.9",
#   "'22 jul 2019' ;	'27 jul 2019'; 	6 	;4 	       ;37.5 	;'25 jul 2019' 	;48.2",
#   "'23 aug 2019' ;	'28 aug 2019'; 	6 	;3 	       ;32.8 	;'27 aug 2019' 	;28.3",
#   "'05 aug 2020' ;	'17 aug 2020'; 	13 	;9 	       ;34.6 	;'08 aug 2020' 	;79.6"]   

#   *Het hittegolfgetal wordt berekend door van de reeks van dagen met een maximumtemperatuur boven de 25 graden alle waarden boven de 25 graden bij elkaar op te tellen. 
#    Een temperatuur van 25,6 °C draagt 0,6 bij aan het hittegolfgetal, een temperatuur van 31,2 °C draagt 6,2 bij. 

In [ ]:
# based on a idea/method from Vermeulen, 2015
# list of heat waves from https://www.knmi.nl/nederland-nu/klimatologie/lijsten/hittegolven:
#   
input={'start':['22 aug 2001',
                '31 jul 2003',
                '02 aug 2004',
                '18 jun 2005',
                '30 jun 2006',
                '15 jul 2006',
                '21 jul 2013',
                '30 jun 2015',
                '15 jul 2018',
                '29 jul 2018',
                '22 jul 2019',
                '23 aug 2019',
                '05 aug 2020'],
       'end(inc.)':['26 aug 2001',
                    '13 aug 2003',
                    '11 aug 2004',
                    '24 jun 2005',
                    '06 jul 2006',
                    '30 jul 2006',
                    '27 jul 2013',
                    '05 jul 2015',
                    '27 jul 2018',
                    '07 aug 2018',
                    '27 jul 2019',
                    '28 aug 2019',
                    '17 aug 2020'],
       'duration(days)':[5, 
                        14,
                        10,
                        7 ,
                        7 ,
                        16,
                        7 ,
                        6 ,
                        13,
                        10,
                        6 ,
                        6 ,
                        13],
       'tropic._days':[3,
                         7,
                         3,
                         3,
                         3,
                         8,
                         3,
                         3,
                         4,
                         4,
                         4,
                         3,
                         9],
       'MaxTemp(C)':[31.1, 
                         35.0,
                         32.5,
                         32.8,
                         32.0,
                         35.7,
                         32.6,
                         33.1,
                         35.7,
                         33.9,
                         37.5,
                         32.8,
                         34.6],
        'MaxTempDate':['25 aug 2001',
                     '07 aug 2003',
                     '09 aug 2004',
                     '20 jun 2005',
                     '04 jul 2006',
                     '19 jul 2006',
                     '22 jul 2013',
                     '01 jul 2015',
                     '26 jul 2018',
                     '07 aug 2018',
                     '25 jul 2019',
                     '27 aug 2019',
                     '08 aug 2020'],
       'HeatwaveNumber':[ 24.0,  #Hittegolf nummer, see *
                      67.4,
                      35.1,
                      28.5,
                      31.7,
                      85.8,
                      27.8,
                      33.5,
                      52.7,
                      40.9,
                      48.2,
                      28.3,
                      79.6]}

df=pd.DataFrame(input)
df.loc[:,'start']=pd.to_datetime(df['start'],format='%d %b %Y')
df.loc[:,'end(inc.)']=pd.to_datetime(df['end(inc.)'],format='%d %b %Y')
df.loc[:,'MaxTempDate']=pd.to_datetime(df['MaxTempDate'],format='%d %b %Y')

df.dtypes
#   *Het hittegolfgetal wordt berekend door van de reeks van dagen met een maximumtemperatuur boven de 25 graden alle waarden boven de 25 graden bij elkaar op te tellen. 
#    Een temperatuur van 25,6 °C draagt 0,6 bij aan het hittegolfgetal, een temperatuur van 31,2 °C draagt 6,2 bij. 

### end of historical heatwave data table creation 

In [ ]:
df